<a href="https://colab.research.google.com/github/Nabil-zr/Heart_Disease_Risk_Prediction_App/blob/main/Heart_Disease_Prediction_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving my_clean_data.csv to my_clean_data.csv


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression, SGDRegressor, LogisticRegression
from sklearn.metrics import root_mean_squared_error, make_scorer, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import pickle
import os

In [ ]:
from sklearn import set_config
set_config(transform_output='pandas')

In [ ]:
import pandas as pd

# Replace 'filename.csv' with the actual name of your uploaded file
data = pd.read_csv('my_clean_data.csv')
data.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3,30,No,Female,55-59,White,Yes,Yes,Very good,5,Yes,No,Yes
1,No,20.34,No,No,Yes,0,0,No,Female,80 or older,White,No,Yes,Very good,7,No,No,No
2,No,26.58,Yes,No,No,20,30,No,Male,65-69,White,Yes,Yes,Fair,8,Yes,No,No
3,No,24.21,No,No,No,0,0,No,Female,75-79,White,No,No,Good,6,No,No,Yes
4,No,23.71,No,No,No,28,0,Yes,Female,40-44,White,No,Yes,Very good,8,No,No,No


In [ ]:
data.shape

(4999, 18)

**DATA CLEANING**

In [ ]:
#01: DATA INFO

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HeartDisease      4999 non-null   object 
 1   BMI               4999 non-null   float64
 2   Smoking           4999 non-null   object 
 3   AlcoholDrinking   4999 non-null   object 
 4   Stroke            4999 non-null   object 
 5   PhysicalHealth    4999 non-null   int64  
 6   MentalHealth      4999 non-null   int64  
 7   DiffWalking       4999 non-null   object 
 8   Sex               4999 non-null   object 
 9   AgeCategory       4999 non-null   object 
 10  Race              4999 non-null   object 
 11  Diabetic          4999 non-null   object 
 12  PhysicalActivity  4999 non-null   object 
 13  GenHealth         4999 non-null   object 
 14  SleepTime         4999 non-null   int64  
 15  Asthma            4999 non-null   object 
 16  KidneyDisease     4999 non-null   object 


In [ ]:
#data.to_csv('modified_data.csv', index=False)


#from google.colab import files
#files.download('modified_data.csv')

In [ ]:
#02: CHECKING FOR NULL VALUES

data.isnull().sum()

,0
HeartDisease,0
BMI,0
Smoking,0
AlcoholDrinking,0
Stroke,0
PhysicalHealth,0
MentalHealth,0
DiffWalking,0
Sex,0
AgeCategory,0


In [ ]:
#04: REMOVING DUPLICATES

data = data.drop_duplicates()
print("Duplicates removed. New shape:", data.shape)

Duplicates removed. New shape: (4999, 18)


In [ ]:
#03: CHEKING FOR DUPLICATED ROWS

duplicates = data.duplicated()

num_duplicates = duplicates.sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


**SETTING UP THE TRAINING DATA**

In [ ]:
y = data.pop("HeartDisease")

In [ ]:
y

,HeartDisease
0,No
1,No
2,No
3,No
4,No
...,...
4994,No
4995,No
4996,No
4997,No


In [ ]:
X= data

In [ ]:
X

,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,16.60,Yes,No,No,3,30,No,Female,55-59,White,Yes,Yes,Very good,5,Yes,No,Yes
1,20.34,No,No,Yes,0,0,No,Female,80 or older,White,No,Yes,Very good,7,No,No,No
2,26.58,Yes,No,No,20,30,No,Male,65-69,White,Yes,Yes,Fair,8,Yes,No,No
3,24.21,No,No,No,0,0,No,Female,75-79,White,No,No,Good,6,No,No,Yes
4,23.71,No,No,No,28,0,Yes,Female,40-44,White,No,Yes,Very good,8,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,18.64,No,No,No,0,0,No,Female,80 or older,White,No,No,Very good,8,No,No,Yes
4995,27.37,Yes,No,No,0,0,No,Male,45-49,Other,No,Yes,Excellent,6,No,No,No
4996,31.89,No,No,No,0,3,No,Female,35-39,Other,No,Yes,Very good,6,No,No,No
4997,32.98,No,No,No,5,0,Yes,Male,60-64,White,No,Yes,Good,6,No,No,No


In [ ]:
# SETTING AND COMBINING X_TRAIN AND y_TRAIN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train
y_train

,HeartDisease
1738,Yes
4943,No
2916,No
1595,Yes
3214,No
...,...
4426,No
466,No
3092,No
3772,No


In [ ]:
X_train


,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
1738,23.06,Yes,No,No,0,0,No,Male,75-79,White,No,Yes,Very good,7,No,No,No
4943,24.39,No,No,No,0,2,No,Female,50-54,White,No,Yes,Very good,6,No,No,No
2916,21.14,Yes,No,No,0,0,No,Female,80 or older,White,No,Yes,Very good,8,No,No,No
1595,30.34,No,No,No,25,0,Yes,Male,65-69,White,Yes,No,Fair,8,No,Yes,Yes
3214,25.83,Yes,No,No,2,4,No,Female,35-39,Black,No,No,Good,6,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,31.57,Yes,No,No,0,20,No,Male,30-34,Hispanic,No,Yes,Excellent,7,No,No,No
466,24.80,No,No,No,0,0,No,Female,80 or older,White,No,Yes,Excellent,8,No,No,No
3092,33.30,Yes,No,No,25,10,Yes,Female,45-49,White,Yes,No,Poor,7,No,No,No
3772,33.50,No,No,No,0,10,No,Female,60-64,White,Yes,Yes,Very good,8,No,Yes,No


**PROCESSING PIPELINES**

In [ ]:
from sklearn import set_config
set_config(transform_output='pandas')


# Preprocessing Pipelines

# Identify numerical and categorical features
num_features = X.select_dtypes(include="number").columns
cat_features = X.select_dtypes(exclude="number").columns

# Numerical pipeline: Impute missing values using the mean and scaling.

num_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

# - Impute missing categorical values with a constant 'N_A'
cat_imputer = SimpleImputer(strategy="constant", fill_value="N_A")

# Ordinal encoding: Features in order
ordinal_features = ['GenHealth']

# Reshape ordinal_categories to have shape (n_features, )
ordinal_categories = [['Very good', 'Fair', 'Good', 'Poor', 'Excellent']]



ordinal_encoder = OrdinalEncoder(categories=ordinal_categories) # Pass the correctly shaped categories

# Identify nominal categorical features (those that will be one-hot encoded)
nominal_features = list(set(cat_features) - set(ordinal_features))

oh_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)


# Combine ordinal and nominal encoders using a column transformer.
cat_encoder = make_column_transformer(
    (ordinal_encoder, ordinal_features),
    (oh_encoder, nominal_features)
)

# Create a categorical pipeline: imputation followed by encoding.
cat_pipe = make_pipeline(cat_imputer, cat_encoder)

# Combine numerical and categorical pipelines into a full preprocessor.
preprocessor = make_column_transformer(
    (num_pipe, num_features),
    (cat_pipe, cat_features)
)

preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime'], dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('columntransformer',
                                                  ColumnTransformer(tr...
                                                                                                 sparse_output=False),
                                                                                   ['DiffWalking',
                                                                                    'Sex',
                                                                                    'SkinCancer',
                                                                                    'PhysicalActivity',
                                                                                    'AgeCategory',
                                                                                    'Asthma',
                                                                                    'Smoking',
                                                                                    'Race',
                                                                                    'KidneyDisease',
                                                                                    'AlcoholDrinking',
                                                                                    'Diabetic',
                                                                                    'Stroke'])]))]),
                                 Index(['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex',
       'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth',
       'Asthma', 'KidneyDisease', 'SkinCancer'],
      dtype='object'))])

**FITTING THE MODEL AND SETTING THE GRIDSEARCH**

In [ ]:
# Combine preprocessing with Logistic Regression

pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('logreg', LogisticRegression(solver='saga', max_iter=1000))
])

# 7. Hyperparameter grid
param_grid = {
    'logreg__penalty': ['l1', 'l2', 'elasticnet'],
    'logreg__C': [0.01, 0.1, 1, 10],
    'logreg__l1_ratio': [0.0, 0.5, 1.0]  # Only used for elasticnet
}

# 8. Grid search with ROC AUC as scoring metric
trained_pipe = GridSearchCV(pipe, param_grid,
                    cv=5,
                    scoring='roc_auc',
                    n_jobs=-1,
                    verbose=1)

**TRAINING THE MODEL**

In [ ]:
trained_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime'], dtype='object')),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        st...
                                                                         Index(['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex',
       'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth',
       'Asthma', 'KidneyDisease', 'SkinCancer'],
      dtype='object'))])),
                                       ('logreg',
                                        LogisticRegression(max_iter=1000,
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'logreg__C': [0.01, 0.1, 1, 10],
                         'logreg__l1_ratio': [0.0, 0.5, 1.0],
                         'logreg__penalty': ['l1', 'l2', 'elasticnet']},
             scoring='roc_auc', verbose=1)

**THE MODEL SCORE**

In [ ]:
best_auc = trained_pipe.best_score_
print(f"Best ROC AUC from cross-validation: {best_auc:.4f}")

Best ROC AUC from cross-validation: 0.8064


**TESTING THE MODEL**

In [ ]:
probs = trained_pipe.best_estimator_.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, probs)
print(f"ROC AUC on test set: {test_auc:.4f}")

ROC AUC on test set: 0.8158


**RESULTS ON TESTING DATA: PROBABILITY FOR HEART DISEASES**

In [ ]:
probs

array([0.16579768, 0.01926648, 0.01406935, 0.07308053, 0.05645006,
       0.07968349, 0.12939339, 0.48079532, 0.11961432, 0.13530669,
       0.07265677, 0.04050526, 0.06281572, 0.02498088, 0.28397808,
       0.10248696, 0.08266692, 0.03134997, 0.0559023 , 0.010897  ,
       0.09579692, 0.05596848, 0.07901998, 0.00544055, 0.00907172,
       0.05139748, 0.01163682, 0.06486607, 0.01906754, 0.01307314,
       0.0789597 , 0.0956118 , 0.11822257, 0.02373155, 0.08411099,
       0.5061621 , 0.00798799, 0.10085571, 0.05279282, 0.15986208,
       0.12872815, 0.10675547, 0.17075807, 0.22702502, 0.13157685,
       0.02572186, 0.02121389, 0.15699363, 0.13330022, 0.32417166,
       0.35220805, 0.46618759, 0.12543892, 0.02297931, 0.06969149,
       0.16503771, 0.0143697 , 0.04575502, 0.13706763, 0.03222821,
       0.11519758, 0.05482143, 0.20655296, 0.07962956, 0.04790083,
       0.09017268, 0.09426411, 0.01889277, 0.02917564, 0.15178489,
       0.06016726, 0.09370227, 0.00730561, 0.04136674, 0.06761

**STORING THE TRAINED PIPELINE - MODEL -**

In [ ]:
# store the trained pipeline

# Create the directory if it doesn't exist

if not os.path.exists('trained_pipe'):
    os.makedirs('trained_pipe')

pickle.dump(trained_pipe,
            open(file='trained_pipe/trained_pipe_LogReg.sav',
                 mode='wb'))

In [ ]:
print(os.getcwd())

/content


**CREATING A FILE OF MY TRAINED MODEL**

In [ ]:
from google.colab import files

# Download the file
files.download('trained_pipe/trained_pipe_LogReg.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**CREATING THE STREAMLIT APP**

In [ ]:
!uv pip install -q streamlit

In [ ]:
# code necessary for Colab only

import os
import time
from IPython.display import display, HTML


def tunnel_prep():
    for f in ("cloudflared-linux-amd64", "logs.txt", "nohup.out"):
        try:
            os.remove(f"/content/{f}")
            print(f"Deleted {f}")
        except FileNotFoundError:
            continue

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
    !chmod +x cloudflared-linux-amd64
    !nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
    url = ""
    while not url:
        time.sleep(1)
        result = !grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1
        if result:
            url = result[0]
    return display(HTML(f'Your tunnel URL <a href="{url}" target="_blank">{url}</a>'))

In [ ]:
# BACKGROUND COLOR

#import os

# Define the path where app.py is saved, usually /content
#base_path = '/content'

# Create .streamlit inside /content
#os.makedirs(os.path.join(base_path, '.streamlit'), exist_ok=True)

# Write config.toml inside .streamlit folder
#with open(os.path.join(base_path, '.streamlit', 'config.toml'), 'w') as f:
#    f.write("""
#[theme]
#base="dark"
#""")

In [ ]:
!ls -la .streamlit

total 12
drwxr-xr-x 2 root root 4096 Jun  8 20:13 .
drwxr-xr-x 1 root root 4096 Jun  8 20:17 ..
-rw-r--r-- 1 root root   21 Jun  8 20:16 config.toml


In [ ]:
!cat /content/.streamlit/config.toml


[theme]
base="dark"


In [ ]:
!ls -la /content

total 37352
drwxr-xr-x 1 root root     4096 Jun  8 20:13 .
drwxr-xr-x 1 root root     4096 Jun  8 20:01 ..
-rw-r--r-- 1 root root     4337 Jun  8 20:12 app.py
-rwxr-xr-x 1 root root 37839075 May 15 17:19 cloudflared-linux-amd64
drwxr-xr-x 4 root root     4096 Jun  5 13:38 .config
-rw-r--r-- 1 root root      260 Jun  8 20:12 logs.txt
-rw-r--r-- 1 root root   362241 Jun  8 20:04 my_clean_data.csv
-rw------- 1 root root     2726 Jun  8 20:12 nohup.out
drwxr-xr-x 1 root root     4096 Jun  5 13:38 sample_data
drwxr-xr-x 2 root root     4096 Jun  8 20:13 .streamlit
drwxr-xr-x 2 root root     4096 Jun  8 20:11 trained_pipe



**Loading the Model into the Web App**

In [ ]:
%%writefile app.py

from sklearn import set_config
set_config(transform_output='pandas')
import streamlit as st
import pandas as pd
import pickle
import os

# Define the path where app.py is saved, usually /content
base_path = '/content'

# Create .streamlit inside /content
os.makedirs(os.path.join(base_path, '.streamlit'), exist_ok=True)

# Write config.toml inside .streamlit folder
with open(os.path.join(base_path, '.streamlit', 'config.toml'), 'w') as f:
    f.write("""
[theme]
base="dark"
""")

trained_pipe = pickle.load(open('/content/trained_pipe/trained_pipe_LogReg.sav', 'rb'))


st.title("Heart Disease Prediction App")

st.write("""
### Project description
This app predicts the risk of heart disease using a machine learning model with an accuracy of 81%.
Please note that this is not an official medical tool but a project developed as part of a data science course.
Always consult a healthcare professional for medical advice.

""")


# Inputs
BMI = st.number_input("BMI (Body Mass Index: weight (kg) / height^2 (m))", min_value=10.0, max_value=60.0, step=0.1)
Smoking = st.selectbox("Do you smoke?", ["Yes", "No"])
AlcoholDrinking = st.selectbox("Do you consume alcohol heavily?", ["Yes", "No"])
Stroke = st.selectbox("Have you had a stroke?", ["Yes", "No"])
PhysicalHealth = st.number_input("Physical Health (days affected in past 30 days)", min_value=0, max_value=30)
MentalHealth = st.number_input("Mental Health (days affected in past 30 days)", min_value=0, max_value=30)
DiffWalking = st.selectbox("Do you have difficulty walking?", ["Yes", "No"])
Sex = st.selectbox("Sex", ["Male", "Female"])
AgeCategory = st.selectbox("Age Category", [
    '18-24', '25-29', '30-34', '35-39', '40-44',
    '45-49', '50-54', '55-59', '60-64', '65-69',
    '70-74', '75-79', '80 or older'
])
Race = st.selectbox("Race", [
    "White", "Black", "Asian", "American Indian/Alaskan Native",
    "Other", "Hispanic"
])
Diabetic = st.selectbox("Are you diabetic?", ["Yes", "No", "No, borderline diabetes", "Yes (during pregnancy)"])
PhysicalActivity = st.selectbox("Do you engage in physical activity?", ["Yes", "No"])
GenHealth = st.selectbox("General Health", ['Very good', 'Fair', 'Good', 'Poor', 'Excellent'])

SleepTime = st.number_input("Average Sleep Time (hours/day)", min_value=0.0, max_value=24.0, step=0.5)
Asthma = st.selectbox("Do you have asthma?", ["Yes", "No"])
KidneyDisease = st.selectbox("Do you have kidney disease?", ["Yes", "No"])
SkinCancer = st.selectbox("Do you have skin cancer?", ["Yes", "No"])


# ----- Make Prediction -----

if st.button("Predict"):
    input_data = pd.DataFrame({
        'BMI': [BMI],
        'Smoking': [Smoking],
        'AlcoholDrinking': [AlcoholDrinking],
        'Stroke': [Stroke],
        'PhysicalHealth': [PhysicalHealth],
        'MentalHealth': [MentalHealth],
        'DiffWalking': [DiffWalking],
        'Sex': [Sex],
        'AgeCategory': [AgeCategory],
        'Race': [Race],
        'Diabetic': [Diabetic],
        'PhysicalActivity': [PhysicalActivity],
        'GenHealth': [GenHealth],
        'SleepTime': [SleepTime],
        'Asthma': [Asthma],
        'KidneyDisease': [KidneyDisease],
        'SkinCancer': [SkinCancer]
    })

    st.write("🧪 Input Data Preview:")
    st.dataframe(input_data)

  # Predict using the pipeline

    prediction = trained_pipe.predict(input_data)




    prob = trained_pipe.predict_proba(input_data)[0][1]  # Probability of heart disease

    if prob > 0.70:
        st.error(f"⚠️ Very High Risk of heart disease! (Probability: {prob:.2%})")
    elif 0.45 < prob <= 0.70:
        st.warning(f"⚠️ Moderate Risk of heart disease. (Probability: {prob:.2%})")
    elif 0 < prob <= 0.45:
        st.info(f"ℹ️ Low Risk detected. (Probability: {prob:.2%})")
    else:
        st.success(f"✅ No risk detected. (Probability: {prob:.2%})")


    #if prediction[0] == 1:
      #st.error(f"⚠️ High risk of heart disease! (Probability: {prob:.2%})")
    #else:
      #st.success(f"✅ Low risk of heart disease. (Probability: {prob:.2%})")

    #prob = trained_pipe.predict_proba(input_data)[0][1]  # Probability of heart disease

    #if prediction[0] == 1:
      #st.error(f"⚠️ High risk of heart disease! (Probability: {prob:.2%})")
    #else:
      #st.success(f"✅ Low risk of heart disease. (Probability: {prob:.2%})")





Overwriting app.py


In [ ]:
tunnel_prep()

Deleted cloudflared-linux-amd64
Deleted logs.txt
Deleted nohup.out
nohup: appending output to 'nohup.out'


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
#!streamlit run app.py &>/content/logs.txt &

In [ ]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

seaborn==0.13.2
pandas==2.2.2


In [ ]:
!pip freeze | grep -E 'streamlit|pandas|scikit-learn|numpy'

geopandas==1.0.1
numpy==2.0.2
pandas==2.2.2
pandas-datareader==0.10.0
pandas-gbq==0.29.0
pandas-stubs==2.2.2.240909
scikit-learn==1.6.1
sklearn-pandas==2.2.0
streamlit==1.45.1
